In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.makedirs('/content/drive/My Drive/Customer-SA-Quantuva', exist_ok=True)

Mounted at /content/drive


In [ ]:
# Install Dependencies

!pip install -q transformers google_play_scraper
!pip install -q pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00


In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews_all,app
import plotly.express as px

In [ ]:
# Scrap Reviews
jsreview = reviews_all('com.jeevansathi.android',
    country = 'in',
    sort = Sort.NEWEST,
    count = 1000)

shreview = reviews_all('com.shaadi.android',
    country = 'in',
    sort = Sort.NEWEST,
    count = 1000)

bmreview = reviews_all('com.bharatmatrimony',
    country = 'in',
    sort = Sort.NEWEST,
    count = 1000)

reviews = [jsreview,shreview,bmreview]

In [ ]:
# Reading Data
DFs = []
for i in reviews:
    DFs.append(pd.json_normalize(i))

In [ ]:
# Cleaning and Data Wrangling
appNames = ['Jeevansathi','Shaadi','BharatMatrimony']
for i,df in enumerate(DFs):
    df['appName'] = appNames[i]

df = pd.concat(DFs)

df = df[df['at']>'2023-01-01']

df = df.reset_index(drop=True)

df = df[ ['appName','content', 'score', 'at']]
df.columns = ['appName','reviewText', 'ratings', 'at']
df = df[df['reviewText'].notna()]

from datetime import datetime
# Tell pandas the existing format of your dates
df['at'] = pd.to_datetime(df['at'], format='%d-%m-%Y')

In [ ]:
df.to_csv('/content/drive/My Drive/Customer-SA-Quantuva/reviewsLarge.csv',
          index=False, escapechar='\\')

In [ ]:
df = df.sample(n=5000).reset_index(drop=True)

In [ ]:
df

,appName,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,Jeevansathi,Useful aap,4,0,38.7.35,2024-04-19 18:29:42,"Hello Shubham,\nWe appreciate your interest in...",2024-04-20 04:58:29,38.7.35
1,Jeevansathi,Good experience 👍,5,0,38.8.8,2024-08-07 15:32:09,None,NaT,38.8.8
2,Jeevansathi,Getting spmmaed request from +60 and other num...,1,1,38.6.3,2023-05-14 16:36:10,"Hello Vikrant,\nWe regret the poor experience....",2023-05-16 04:41:28,38.6.3
3,Jeevansathi,Excellent,5,0,38.6.12,2023-07-09 15:39:21,None,NaT,38.6.12
4,BharatMatrimony,Very bad service ... don't use any one worst e...,1,0,10.5,2024-05-11 10:11:05,"Hello Raviraj, we apologize for the inconvenie...",2024-05-25 10:22:06,10.5
...,...,...,...,...,...,...,...,...,...
4995,BharatMatrimony,easy and good software,5,1,9.7,2023-11-28 14:03:31,"Hello Muthu ganesan, that's great! Thanks for ...",2023-11-29 04:51:50,9.7
4996,BharatMatrimony,Good,5,0,9.0,2023-04-04 07:23:15,"Hello Kavinn, you made our day with your ratin...",2023-04-05 03:26:20,9.0
4997,Jeevansathi,Good,4,0,38.7.2,2023-09-17 14:00:41,Hi! We are glad that you like our app. We'd ve...,2023-09-19 12:13:33,38.7.2
4998,Shaadi,There is no any input search bar option in app...,1,0,9.39.1,2023-04-24 16:57:57,"Hi there, we would like to have a word with yo...",2023-04-25 06:46:34,9.39.1


In [ ]:
# Importing Model
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

SA_pipeline = pipeline("sentiment-analysis",
                       model="siebert/sentiment-roberta-large-english",
                       truncation=True)

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
# Get Sentiment Analysis
result = pd.Series(df['content'].apply(lambda x: SA_pipeline(x)),index=df.index)

In [ ]:
df['sentiment'] = result.apply(lambda x: x[0]['label'])
df['sentiment_score'] = result.apply(lambda x: x[0]['score'])

In [ ]:
df

,appName,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,sentiment_score
0,Jeevansathi,Useful aap,4,0,38.7.35,2024-04-19 18:29:42,"Hello Shubham,\nWe appreciate your interest in...",2024-04-20 04:58:29,38.7.35,POSITIVE,0.997356
1,Jeevansathi,Good experience 👍,5,0,38.8.8,2024-08-07 15:32:09,None,NaT,38.8.8,POSITIVE,0.998867
2,Jeevansathi,Getting spmmaed request from +60 and other num...,1,1,38.6.3,2023-05-14 16:36:10,"Hello Vikrant,\nWe regret the poor experience....",2023-05-16 04:41:28,38.6.3,NEGATIVE,0.999435
3,Jeevansathi,Excellent,5,0,38.6.12,2023-07-09 15:39:21,None,NaT,38.6.12,POSITIVE,0.998581
4,BharatMatrimony,Very bad service ... don't use any one worst e...,1,0,10.5,2024-05-11 10:11:05,"Hello Raviraj, we apologize for the inconvenie...",2024-05-25 10:22:06,10.5,NEGATIVE,0.999497
...,...,...,...,...,...,...,...,...,...,...,...
4995,BharatMatrimony,easy and good software,5,1,9.7,2023-11-28 14:03:31,"Hello Muthu ganesan, that's great! Thanks for ...",2023-11-29 04:51:50,9.7,POSITIVE,0.998882
4996,BharatMatrimony,Good,5,0,9.0,2023-04-04 07:23:15,"Hello Kavinn, you made our day with your ratin...",2023-04-05 03:26:20,9.0,POSITIVE,0.998415
4997,Jeevansathi,Good,4,0,38.7.2,2023-09-17 14:00:41,Hi! We are glad that you like our app. We'd ve...,2023-09-19 12:13:33,38.7.2,POSITIVE,0.998415
4998,Shaadi,There is no any input search bar option in app...,1,0,9.39.1,2023-04-24 16:57:57,"Hi there, we would like to have a word with yo...",2023-04-25 06:46:34,9.39.1,NEGATIVE,0.998466


In [ ]:
df.to_csv('/content/drive/My Drive/Customer-SA-Quantuva/reviews_sentiment.csv',
          index=False, escapechar='\\')